### Test out the new logger function

First test was to make sure that the logger only initiates (or at the very least makes a message) only once. Had to make the logger metaclass=Singleton. Totally don't understand that. 

In [3]:
from fireatlas import postprocess, settings
from fireatlas.FireLog import logger
import os

Now let's test out it's ability to change the path of the file handler. Make its subdirectories defined by year and region. 

In [2]:
# in a typical script, there should be region and year already defined as global vars
# define them here
region = ['Zogg', 'Zogg_perimeter']
tst = [2020, 9, 15, 'AM']
location = None


Show it's file handler path intially, change it to a subdirectory, reset it.

In [4]:
print(logger.handlers)
newpath = postprocess.all_dir(tst, region, location)
logger.update_fh_dir(newpath)
print(logger.handlers)
logger.reset_fh()
print(logger.handlers)


[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\running.log (INFO)>]
[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\data\FEDSoutput-v3\Zogg\2020\running.log (INFO)>]
[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\running.log (INFO)>]


Next, need to show that the logger will change file handler path when the region and year changes. I suspect this will have no issues when done in serial, but question remains when done in parallel with `dask`. 

In [5]:
# create a list of "fires" to process
region_list = [['castle', 'castle_perim'], 
               ['caldor', 'caldor_perim'], 
               ['mosquito', 'mosquito_perim'], 
               ['zogg', 'zogg_perim']]
tst_list = [
    [2020, 8], [2021, 7], [2022, 8], [2020, 9]
]
location = None

In [6]:
# create a function to apply to each iteration
# does the logger properly log in the right subdirectory
# can it also log to the main directory when it's not region/year specific? 
def test_logger(logger, tst, region, location):
    # log into the main file handler
    logger.reset_fh()
    logger.info("Processing something for all fires.")

    # log into a subdirectory
    subdir = postprocess.all_dir(tst, region, location)
    logger.update_fh_dir(subdir)
    os.makedirs(subdir, exist_ok=True)
    logger.info(f"logging into the {region[0]} fire.")
    

Logger works great when used in serial. Check out the running.log files in the data directory.

In [7]:
for tst, region in zip(tst_list, region_list):
    test_logger(logger, tst, region, location)

2024-08-13 13:45:47,162 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 13:45:47,167 - fireatlas.FireLog - INFO - logging into the castle fire.
2024-08-13 13:45:47,172 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 13:45:47,175 - fireatlas.FireLog - INFO - logging into the caldor fire.
2024-08-13 13:45:47,182 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 13:45:47,188 - fireatlas.FireLog - INFO - logging into the mosquito fire.
2024-08-13 13:45:47,190 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-13 13:45:47,198 - fireatlas.FireLog - INFO - logging into the zogg fire.


Test out the logger when using `dask`!

In [8]:
from dask.distributed import Client
from functools import partial
import logging

# no idea how to avoid doing this
all_loggers = logging.root.manager.loggerDict
for logger_name, logger in all_loggers.items():
    if logger_name.startswith('fireatlas') and isinstance(logger, logging.Logger):
        logger.setLevel(logging.ERROR)

# start dask client
client = Client(n_workers = 4, threads_per_worker = 1)

print(client.dashboard_link)

http://127.0.0.1:8787/status


In [10]:
# try it out 
futures = client.map(partial(test_logger, logger, location=location), tst_list, region_list)
results = client.gather(futures)

In [11]:
client.close()

### Test on FEDS

It's one thing to test on isolated logging. Now let's test it with running FEDS. 

First, without changing the logging path, works fine. 

In [4]:
from fireatlas import FireRun
perim_path = os.path.join(settings.dirextdata, 'perimeters', 'zogg.fgb')
FireRun.constrainByShape_Run(perim_path)

2024-08-14 09:24:37,466 - fireatlas.FireLog - INFO - Preprocessing has already occurred for this landcover file.
2024-08-14 09:24:37,467 - fireatlas.FireLog - INFO - func:preprocess_landcover took: 1.24 ms
2024-08-14 09:24:37,811 - fireatlas.FireLog - INFO - =============== Running: ZOGG_00009978 ===============
2024-08-14 09:24:38,075 - fireatlas.FireLog - INFO - func:preprocess_region took: 264.35 ms
2024-08-14 09:24:38,075 - fireatlas.FireLog - INFO - preprocessing VIIRS\VNP14IMGML\VNP14IMGML.202009.C1.05.txt


Processing days: 100%|██████████| 30/30 [00:28<00:00,  1.04it/s]

2024-08-14 09:25:13,825 - fireatlas.FireLog - INFO - func:preprocess_input_file took: 35.75 sec
2024-08-14 09:25:13,825 - fireatlas.FireLog - INFO - preprocessing VIIRS\VNP14IMGML\VNP14IMGML.202010.C1.05.txt



Processing days: 100%|██████████| 31/31 [00:18<00:00,  1.67it/s]

2024-08-14 09:25:37,673 - fireatlas.FireLog - INFO - func:preprocess_input_file took: 23.85 sec
2024-08-14 09:25:37,709 - fireatlas.FireLog - INFO - func:read_region took: 36.45 ms
2024-08-14 09:25:37,713 - fireatlas.FireLog - INFO - func:read_region took: 3.86 ms
2024-08-14 09:25:37,713 - fireatlas.FireLog - INFO - filtering and clustering 2020-9-27 AM, SNPP, ZOGG_00009978
2024-08-14 09:25:37,741 - fireatlas.FireLog - INFO - func:read_preprocessed_input took: 27.49 ms
2024-08-14 09:25:37,775 - fireatlas.FireLog - INFO - func:preprocess_region_t took: 65.51 ms
2024-08-14 09:25:37,791 - fireatlas.FireLog - INFO - func:read_region took: 16.49 ms


2024-08-14 09:25:37,791 - fireatlas.FireLog - INFO - filtering and clustering 2020-9-27 PM, SNPP, ZOGG_00009978
2024-08-14 09:25:37,896 - fireatlas.FireLog - INFO - func:read_preprocessed_input took: 104.73 ms
2024-08-14 09:25:37,907 - fireatlas.FireLog - INFO - func:preprocess_region_t took: 132.53 ms
2024-08-14 09:25:37,925 - fireatlas.FireLog - INFO - func:read_region took: 17.78 ms
2024-08-14 09:25:37,926 - fireatlas.FireLog - INFO - filtering and clustering 2020-9-28 AM, SNPP, ZOGG_00009978
2024-08-14 09:25:37,959 - fireatlas.FireLog - INFO - func:read_preprocessed_input took: 31.46 ms
2024-08-14 09:25:38,352 - fireatlas.FireLog - INFO - func:do_clustering took: 37.59 ms
2024-08-14 09:25:38,384 - fireatlas.FireLog - INFO - func:preprocess_region_t took: 476.67 ms
2024-08-14 09:25:38,393 - fireatlas.FireLog - INFO - func:read_region took: 8.35 ms
2024-08-14 09:25:38,393 - fireatlas.FireLog - INFO - filtering and clustering 2020-9-28 PM, SNPP, ZOGG_00009978
2024-08-14 09:25:38,463 -

Next, I want to make any logs that are specific to a fire go into the FEDSoutput subdirectories. I'll need to modify the logging configurations in `FireMain` and `postprocess`. Also, delete any postprocessed and final outputs from this fire to start from a clean slate. 